FLUX_1_dev_LoRA_Training.ipynb

# AI Toolkit by Ostris
## FLUX.1 Training


In [ ]:
!git clone https://github.com/ostris/ai-toolkit
!mkdir -p /content/dataset

Cloning into 'ai-toolkit'...
remote: Enumerating objects: 3490, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 3490 (delta 216), reused 274 (delta 197), pack-reused 3186 (from 1)
Receiving objects: 100% (3490/3490), 29.40 MiB | 37.96 MiB/s, done.
Resolving deltas: 100% (2662/2662), done.


Put your image dataset in the `/content/dataset` folder

In [ ]:
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt


Submodule 'repositories/batch_annotator' (https://github.com/ostris/batch-annotator) registered for path 'repositories/batch_annotator'
Submodule 'repositories/ipadapter' (https://github.com/tencent-ailab/IP-Adapter.git) registered for path 'repositories/ipadapter'
Submodule 'repositories/leco' (https://github.com/p1atdev/LECO) registered for path 'repositories/leco'
Submodule 'repositories/sd-scripts' (https://github.com/kohya-ss/sd-scripts.git) registered for path 'repositories/sd-scripts'
Cloning into '/content/ai-toolkit/repositories/batch_annotator'...
Cloning into '/content/ai-toolkit/repositories/ipadapter'...
Cloning into '/content/ai-toolkit/repositories/leco'...
Cloning into '/content/ai-toolkit/repositories/sd-scripts'...
Submodule path 'repositories/batch_annotator': checked out '420e142f6ad3cc14b3ea0500affc2c6c7e7544bf'
Submodule 'repositories/controlnet' (https://github.com/lllyasviel/ControlNet-v1-1-nightly.git) registered for path 'repositories/batch_annotator/repositor

## Model License
Training currently only works with FLUX.1-dev. Which means anything you train will inherit the non-commercial license. It is also a gated model, so you need to accept the license on HF before using it. Otherwise, this will fail. Here are the required steps to setup a license.

Sign into HF and accept the model access here [black-forest-labs/FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev)

[Get a READ key from huggingface](https://huggingface.co/settings/tokens/new?) and place it in the next cell after running it.

In [ ]:
import getpass
import os

# Prompt for the token
hf_token = getpass.getpass('Enter your HF access token and press enter: ')

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

Enter your HF access token and press enter: ··········
HF_TOKEN environment variable has been set.


In [ ]:
import os
import sys
sys.path.append('/content/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
#!pip install pillow-heif



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0


In [ ]:
# import os
# from PIL import Image
# import pillow_heif

# def convert_heic_to_jpeg(directory):
#     pillow_heif.register_heif_opener()

#     for filename in os.listdir(directory):
#         if filename.lower().endswith('.jpeg') or filename.lower().endswith('.jpg'):
#             filepath = os.path.join(directory, filename)
#             try:
#                 # Try to open as HEIC
#                 with Image.open(filepath) as img:
#                     # If successful, save as JPEG
#                     jpeg_filename = os.path.splitext(filename)[0] + '_converted.jpg'
#                     jpeg_filepath = os.path.join(directory, jpeg_filename)
#                     img.save(jpeg_filepath, 'JPEG')
#                     print(f"Converted {filename} to JPEG: {jpeg_filename}")
#             except Exception as e:
#                 print(f"Failed to convert {filename}: {str(e)}")

# # Usage
# convert_heic_to_jpeg('/content/dataset')

Converted IMG_3044.jpeg to JPEG: IMG_3044_converted.jpg
Converted IMG_2898.jpeg to JPEG: IMG_2898_converted.jpg
Converted IMG_2916.jpeg to JPEG: IMG_2916_converted.jpg
Converted IMG_2865.jpeg to JPEG: IMG_2865_converted.jpg
Converted IMG_2948.jpeg to JPEG: IMG_2948_converted.jpg
Converted IMG_2971.jpeg to JPEG: IMG_2971_converted.jpg
Converted IMG_2896.jpeg to JPEG: IMG_2896_converted.jpg
Converted IMG_2874.jpeg to JPEG: IMG_2874_converted.jpg
Converted IMG_2920.jpeg to JPEG: IMG_2920_converted.jpg
Converted IMG_2879.jpeg to JPEG: IMG_2879_converted.jpg
Converted IMG_2989.jpeg to JPEG: IMG_2989_converted.jpg


## Setup

This is your config. It is documented pretty well. Normally you would do this as a yaml file, but for colab, this will work. This will run as is without modification, but feel free to edit as you want.

In [ ]:
from collections import OrderedDict

job_to_run = OrderedDict([
    ('job', 'extension'),
    ('config', OrderedDict([
        # this name will be the folder and filename name
        ('name', 'my_first_flux_lora_v1'),
        ('process', [
            OrderedDict([
                ('type', 'sd_trainer'),
                # root folder to save training sessions/samples/weights
                ('training_folder', '/content/output'),
                # uncomment to see performance stats in the terminal every N steps
                #('performance_log_every', 1000),
                ('device', 'cuda:0'),
                # if a trigger word is specified, it will be added to captions of training data if it does not already exist
                # alternatively, in your captions you can add [trigger] and it will be replaced with the trigger word
                # ('trigger_word', 'image'),
                ('network', OrderedDict([
                    ('type', 'lora'),
                    ('linear', 16),
                    ('linear_alpha', 16)
                ])),
                ('save', OrderedDict([
                    ('dtype', 'float16'),  # precision to save
                    ('save_every', 250),  # save every this many steps
                    ('max_step_saves_to_keep', 4)  # how many intermittent saves to keep
                ])),
                ('datasets', [
                    # datasets are a folder of images. captions need to be txt files with the same name as the image
                    # for instance image2.jpg and image2.txt. Only jpg, jpeg, and png are supported currently
                    # images will automatically be resized and bucketed into the resolution specified
                    OrderedDict([
                        ('folder_path', '/content/dataset'),
                        ('caption_ext', 'txt'),
                        ('caption_dropout_rate', 0.05),  # will drop out the caption 5% of time
                        ('shuffle_tokens', False),  # shuffle caption order, split by commas
                        ('cache_latents_to_disk', True),  # leave this true unless you know what you're doing
                        ('resolution', [512, 768, 1024])  # flux enjoys multiple resolutions
                    ])
                ]),
                ('train', OrderedDict([
                    ('batch_size', 1),
                    ('steps', 4000),  # total number of steps to train 500 - 4000 is a good range
                    ('gradient_accumulation_steps', 1),
                    ('train_unet', True),
                    ('train_text_encoder', False),  # probably won't work with flux
                    ('content_or_style', 'balanced'),  # content, style, balanced
                    ('gradient_checkpointing', True),  # need the on unless you have a ton of vram
                    ('noise_scheduler', 'flowmatch'),  # for training only
                    ('optimizer', 'adamw8bit'),
                    ('lr', 4e-4),
                    # uncomment this to skip the pre training sample
                    #('skip_first_sample', True),

                    # ema will smooth out learning, but could slow it down. Recommended to leave on.
                    ('ema_config', OrderedDict([
                        ('use_ema', True),
                        ('ema_decay', 0.99)
                    ])),

                    # will probably need this if gpu supports it for flux, other dtypes may not work correctly
                    ('dtype', 'bf16')
                ])),
                ('model', OrderedDict([
                    # huggingface model name or path
                    ('name_or_path', 'black-forest-labs/FLUX.1-dev'),
                    ('is_flux', True),
                    ('quantize', True),  # run 8bit mixed precision
                    #('low_vram', True),  # uncomment this if the GPU is connected to your monitors. It will use less vram to quantize, but is slower.
                ])),
                ('sample', OrderedDict([
                    ('sampler', 'flowmatch'),  # must match train.noise_scheduler
                    ('sample_every', 250),  # sample every this many steps
                    ('width', 1024),
                    ('height', 1024),
                    ('prompts', [
                        # you can add [trigger] to the prompts here and it will be replaced with the trigger word
                        #'[trigger] holding a sign that says \'I LOVE PROMPTS!\'',
                        'man with red hair, playing chess at the park, bomb going off in the background',
                        'a man holding a coffee cup, in a beanie, sitting at a cafe',
                        'a horse is a DJ at a night club, fish eye lens, smoke machine, lazer lights, holding a martini',
                        'a man showing off his cool new t shirt at the beach, a shark is jumping out of the water in the background',
                        'a bear building a log cabin in the snow covered mountains'

                    ]),
                    ('neg', ''),  # not used on flux
                    ('seed', 42),
                    ('walk_seed', True),
                    ('guidance_scale', 4),
                    ('sample_steps', 20)
                ]))
            ])
        ])
    ])),
    # you can add any additional meta info here. [name] is replaced with config name at top
    ('meta', OrderedDict([
        ('name', '[name]'),
        ('version', '1.0')
    ]))
])


## Run it

Below does all the magic. Check your folders to the left. Items will be in output/LoRA/your_name_v1 In the samples folder, there are preiodic sampled. This doesnt work great with colab. They will be in /content/output

In [ ]:
run_job(job_to_run)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_5m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/usr/local/lib/python3.10/dist-packages/controlnet_aux/segment_anything/modeling/tiny_vit_sam.py:654: UserWarning: Overwriting tiny_vit_11m_224 in registry with controlnet_aux.segment_anything.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.

{
    "type": "sd_trainer",
    "training_folder": "/content/output",
    "device": "cuda:0",
    "network": {
        "type": "lora",
        "linear": 16,
        "linear_alpha": 16
    },
    "save": {
        "dtype": "float16",
        "save_every": 250,
        "max_step_saves_to_keep": 4
    },
    "datasets": [
        {
            "folder_path": "/content/dataset",
            "caption_ext": "txt",
            "caption_dropout_rate": 0.05,
            "shuffle_tokens": false,
            "cache_latents_to_disk": true,
            "resolution": [
                512,
                768,
                1024
            ]
        }
    ],
    "train": {
        "batch_size": 1,
        "steps": 4000,
        "gradient_accumulation_steps": 1,
        "train_unet": true,
        "train_text_encoder": false,
        "content_or_style": "balanced",
        "gradient_checkpointing": true,
        "noise_scheduler": "flowmatch",
        "optimizer": "adamw8bit",
        "lr": 0.0004

transformer/config.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

...

Saving at step 3750


my_first_flux_lora_v1:  94%|█████████▎| 3749/4000 [2:20:31<09:00,  2.15s/it, lr: 4.0e-04 loss: 3.221e-01]

Saved to /content/output/my_first_flux_lora_v1/optimizer.pt
Removing old save: /content/output/my_first_flux_lora_v1/my_first_flux_lora_v1_000002750.safetensors


my_first_flux_lora_v1: 100%|█████████▉| 3999/4000 [2:29:49<00:02,  2.25s/it, lr: 4.0e-04 loss: 1.987e-01]



Saved to /content/output/my_first_flux_lora_v1/optimizer.pt


## Done

Check your ourput dir and get your slider


In [ ]:
!zip -r name_for_zipped_file.zip /content/output/my_first_flux_lora_v1/samples

  adding: content/output/my_first_flux_lora_v1/samples/ (stored 0%)
  adding: content/output/my_first_flux_lora_v1/samples/1723964626246__000002500_0.jpg (deflated 0%)
  adding: content/output/my_first_flux_lora_v1/samples/1723966594329__000003250_0.jpg (deflated 1%)
  adding: content/output/my_first_flux_lora_v1/samples/1723968591094__000004000_2.jpg (deflated 1%)
  adding: content/output/my_first_flux_lora_v1/samples/1723967246611__000003500_0.jpg (deflated 0%)
  adding: content/output/my_first_flux_lora_v1/samples/1723958739399__000000250_1.jpg (deflated 1%)
  adding: content/output/my_first_flux_lora_v1/samples/1723965988812__000003000_3.jpg (deflated 2%)
  adding: content/output/my_first_flux_lora_v1/samples/1723967263750__000003500_1.jpg (deflated 1%)
  adding: content/output/my_first_flux_lora_v1/samples/1723959431343__000000500_3.jpg (deflated 1%)
  adding: content/output/my_first_flux_lora_v1/samples/1723967315152__000003500_4.jpg (deflated 0%)
  adding: content/output/my_firs

In [ ]:
/content/dataset